In [1]:
%run ./NB_startup.ipynb

Starting Spark App: spark_session_dc0376a3


25/12/09 22:40:45 WARN Utils: Your hostname, Souravs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.102 instead (on interface en0)
25/12/09 22:40:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/souravagasti/.ivy2/cache
The jars for the packages stored in: /Users/souravagasti/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
org.apache.spark#spark-token-provider-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a4dbe804-6d5a-42a6-9b9b-35331a498ae2;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-

:: loading settings :: url = jar:file:/Users/souravagasti/projects/real-time-data/venv-realtime/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.5.0 in central
	found com.github.luben#zstd-jni;1.5.5-1 in central
	found org.lz4#lz4-java;1.8.0 in central
:: resolution report :: resolve 170ms :: artifacts dl 5ms
	:: modules in use:
	com.github.luben#zstd-jni;1.5.5-1 from central in [default]
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.com

Spark session initialized successfully with Delta support.


In [2]:
topicname = "nyctaxistream"

generate and write nyctaxi streams to kafka asynchronously for two dispatching_base_num values

In [3]:
number_of_events = 20
desired_latency = 0

B03494_stream_task = asyncio.create_task(write_to_kafka_async(topicname, generate_nyctaxistream_async(number_of_events, desired_latency,dispatching_base_num=["B03494"])))
B03496_stream_task =  asyncio.create_task(write_to_kafka_async(topicname, generate_nyctaxistream_async(number_of_events, desired_latency,dispatching_base_num=["B03496"])))

await asyncio.gather(B03494_stream_task, B03496_stream_task) #simultaneous execution of both streams

Generated: {'hvfhs_license_num': 'HV00001', 'dispatching_base_num': 'B03494', 'originating_base_num': 'B02764', 'request_datetime': '2025-01-01 11:10:45', 'on_scene_datetime': '2025-01-01 09:39:45', 'pickup_datetime': '2025-01-01 11:33:29', 'dropoff_datetime': '2025-01-01 06:06:45', 'PULocationID': 17, 'DOLocationID': 4, 'trip_miles': 13.0, 'trip_time': 6538}
Generated: {'hvfhs_license_num': 'HV00004', 'dispatching_base_num': 'B03496', 'originating_base_num': None, 'request_datetime': '2025-01-01 16:35:20', 'on_scene_datetime': '2025-01-01 20:52:09', 'pickup_datetime': '2025-01-01 13:02:42', 'dropoff_datetime': '2025-01-01 21:50:55', 'PULocationID': 122, 'DOLocationID': 15, 'trip_miles': 17.03, 'trip_time': 5843}
Generated: {'hvfhs_license_num': 'HV00006', 'dispatching_base_num': 'B03494', 'originating_base_num': 'B03406', 'request_datetime': '2025-01-01 05:04:27', 'on_scene_datetime': '2025-01-01 14:14:56', 'pickup_datetime': '2025-01-01 09:42:32', 'dropoff_datetime': '2025-01-01 06:1

[None, None]

In [5]:
# Check if Delta table exists
# If not, read raw parquet data and write as Delta table
try:
    if not(DeltaTable.isDeltaTable(spark, delta_path)):
        required_cols = [
            "hvfhs_license_num",
            "dispatching_base_num",
            "originating_base_num",
            "request_datetime",
            "on_scene_datetime",
            "pickup_datetime",
            "dropoff_datetime",
            "PULocationID",
            "DOLocationID",
            "trip_miles",
            "trip_time"
        ]

        df_raw = read_spark_parquet(spark, raw_file_path).select(*required_cols)
        df_raw.write.format("delta").mode("overwrite").save(delta_path)
        print(f"Delta table created at {delta_path}")
    else:
        print(f"Delta table already exists at {delta_path}")
except Exception as e:
    print(f"Error checking/creating Delta table: {e}")

finally:
    # Register the Delta table as a temporary view
    spark.read.format("delta").load(delta_path).createOrReplaceTempView("delta_table")

Delta table created at /Users/souravagasti/projects/real-time-data/data/nyctaxi/delta


In [6]:
# spark.sql("delete from delta_table where dispatching_base_num IN ('B03494', 'B03496')")

In [ ]:
#startingOffset = Earliest to read from beginning
params_read_from_kafka = {
    "spark": spark,
    "topic": topicname,
    "schema": schema,
    "startingOffsets": "earliest"
}

df_stream = read_from_kafka(**params_read_from_kafka)

df_stream = (
    df_stream
    .withColumn("request_datetime", to_timestamp("request_datetime"))
    .withColumn("on_scene_datetime", to_timestamp("on_scene_datetime"))
    .withColumn("pickup_datetime", to_timestamp("pickup_datetime"))
    .withColumn("dropoff_datetime", to_timestamp("dropoff_datetime"))
)

Those new records generated in kafka should be caught in this data stream. 
To read from it, write it to memory and see the contents

In [8]:
# write to memory
params_write_spark_stream = {
"parsed_df": df_stream,
"spark": spark,
"topic": "nycstream",       #the in memory df would be of this name
"write_to": "memory",
"trigger_mode": "once",
"output_path": delta_path,
"output_mode": "append",
"merge_keys": merge_keys,
"interval_seconds": 5,
}

query = write_spark_stream(**params_write_spark_stream)

# Wait until the query finishes
try:
    query.awaitTermination()
    # if params_write_spark_stream["trigger_mode"] == "continuous": 
    #     spark.sql("select * from nycstream").show(10)
finally:
    if params_write_spark_stream["trigger_mode"] != "continuous":
        query.stop()

inspect values of stream

In [9]:
spark.sql("select distinct dispatching_base_num from nycstream").show(10)

+--------------------+
|dispatching_base_num|
+--------------------+
|              B03496|
|              B03494|
+--------------------+



lets write this stream to delta. first lets check if these records exist on the delta table

In [10]:
# spark.sql("delete from delta_table where dispatching_base_num IN ('B03494', 'B03496')")

count = spark.sql("""
    SELECT COUNT(*) AS cnt
    FROM delta_table
    WHERE dispatching_base_num IN ('B03494', 'B03496')
""").collect()[0]["cnt"]

print("Count:", count)

Count: 0


lets merge the stream into delta

In [ ]:
params_write_spark_stream = {
"parsed_df": df_stream,
"spark": spark,
"topic": "nycstream",
"write_to": "delta",
"trigger_mode": "once",
"output_path": delta_path,
"output_mode": "append",
"merge_keys": merge_keys,
"interval_seconds": 5,
}

query = write_spark_stream(**params_write_spark_stream)
print(query.lastProgress)
#merge these in!

try:
    query.awaitTermination()
finally:
    if params_write_spark_stream["trigger_mode"] != "continuous":
        query.stop()

Started merge stream with foreachBatch.
None
🔥 MERGE BATCH CALLED. ROWS: 40  EPOCH: 0


Exception in thread Thread-24 (bg_main):
Traceback (most recent call last):
  File "/var/folders/wl/vmxlmyj15pg7hvwhyg4y29540000gn/T/ipykernel_43836/1523115453.py", line 17, in bg_main
    output.update({"application/vnd.vscode.bg.execution.8.result": do_implementation()}, raw=True)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/souravagasti/projects/real-time-data/venv-realtime/lib/python3.13/site-packages/IPython/core/display_functions.py", line 354, in update
    update_display(obj, display_id=self.display_id, **kwargs)
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/souravagasti/projects/real-time-data/venv-realtime/lib/python3.13/site-packages/IPython/core/display_functions.py", line 306, in update_display
    display(obj, display_id=display_id, **kwargs)
    ~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/souravagasti/projects/real-time-data/venv-realtime/lib/python3.13/si

In [12]:

count = spark.sql("""
    SELECT COUNT(*) AS cnt
    FROM delta_table
    WHERE dispatching_base_num IN ('B03494', 'B03496')
""").collect()[0]["cnt"]

print("Count:", count)


Count: 40


In [16]:
spark.sql("select * from nycstream").show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+
|          HV00001|              B03496|                NULL|2025-01-01 02:25:16|2025-01-01 18:59:52|2025-01-01 19:19:31|2025-01-01 03:52:55|         151|          79|     25.77|     1144|
|          HV00003|              B03496|              B03404|2025-01-01 22:55:24|2025-01-01 18:59:52|2025-01-01 11:23:57|2025-01-01 03:59:58|          21|         163|     20.79|     6301|
|          HV00009|              B03496|              B

##### tumbling windows and watermarking

###### window size 15 mins, watermarking 2 mins, max temp, number of drops

In [ ]:
from datetime import datetime

clean_reset_kafka_topics(["location_temperature","nyctaxistream"]) #recreate these two topics

location_temperature_task_1 = asyncio.create_task(write_to_kafka_async("location_temperature", generate_locationid_temperature_async(
    latency = 1, locationId=[258],record_datetime=[datetime(2025, 1, 1, 2, 25, 16)]
    )))

location_temperature_task_2 = asyncio.create_task(write_to_kafka_async("location_temperature", generate_locationid_temperature_async(
    latency = 1, locationId=[258],record_datetime=[datetime(2025, 1, 1, 2, 27, 0)]
    )))

location_temperature_task_3 = asyncio.create_task(write_to_kafka_async("location_temperature", generate_locationid_temperature_async(
    latency = 1, locationId=[258],record_datetime=[datetime(2025, 1, 1, 2, 30, 10)]
    )))

location_temperature_task_4 = asyncio.create_task(write_to_kafka_async("location_temperature", generate_locationid_temperature_async(
    latency = 1, locationId=[258],record_datetime=[datetime(2025, 1, 1, 2, 30, 10)]
    )))

location_temperature_task_5 = asyncio.create_task(write_to_kafka_async("location_temperature", generate_locationid_temperature_async(
    latency = 1, locationId=[258],record_datetime=[datetime(2025, 1, 1, 2, 30, 10)]
    )))

nyctaxi_task_1 = asyncio.create_task(write_to_kafka_async("nyctaxistream", generate_nyctaxistream_async(
    latency = 1, request_datetime = [datetime(2025, 1, 1, 2, 25, 16)],DOLocationID=[258])))

nyctaxi_task_2 = asyncio.create_task(write_to_kafka_async("nyctaxistream", generate_nyctaxistream_async(
    latency = 1, request_datetime = [datetime(2025, 1, 1, 2, 25, 16)],DOLocationID=[258])))

nyctaxi_task_3 = asyncio.create_task(write_to_kafka_async("nyctaxistream", generate_nyctaxistream_async(
    latency = 1, request_datetime = [datetime(2025, 1, 1, 2, 25, 16)],DOLocationID=[258])))

nyctaxi_task_4 = asyncio.create_task(write_to_kafka_async("nyctaxistream", generate_nyctaxistream_async(
    latency = 1, request_datetime = [datetime(2025, 1, 1, 2, 25, 16)],DOLocationID=[258])))

nyctaxi_task_5 = asyncio.create_task(write_to_kafka_async("nyctaxistream", generate_nyctaxistream_async(
    latency = 1, request_datetime = [datetime(2025, 1, 1, 2, 25, 16)],DOLocationID=[258])))


await asyncio.gather(location_temperature_task_1,location_temperature_task_2,location_temperature_task_3,
               location_temperature_task_4,location_temperature_task_5,nyctaxi_task_1,nyctaxi_task_2,
               nyctaxi_task_3,nyctaxi_task_4,nyctaxi_task_5)

Resetting kafka topics...
Deleting topic location_temperature
Topic 'location_temperature' deleted
Deleting topic nyctaxistream
Topic 'nyctaxistream' deleted
Creating Kafka topic location_temperature...
Topic 'location_temperature' created
Creating Kafka topic nyctaxistream...
Topic 'nyctaxistream' created
Kafka topics reset complete.
Generated: {'locationId': 258, 'temperature': 27, 'record_datetime': '2025-01-01 02:25:16'}
Generated: {'locationId': 258, 'temperature': 20, 'record_datetime': '2025-01-01 02:27:00'}
Generated: {'locationId': 258, 'temperature': 24, 'record_datetime': '2025-01-01 02:30:10'}
Generated: {'locationId': 258, 'temperature': 26, 'record_datetime': '2025-01-01 02:30:10'}
Generated: {'locationId': 258, 'temperature': 11, 'record_datetime': '2025-01-01 02:30:10'}
Generated: {'hvfhs_license_num': 'HV00006', 'dispatching_base_num': 'B03288', 'originating_base_num': None, 'request_datetime': '2025-01-01 02:25:16', 'on_scene_datetime': '2025-01-01 02:53:49', 'pickup_

[None, None, None, None, None, None, None, None, None, None]

In [ ]:
# on every 5 mins, with watermarking of 2 mins, record lowest temperature, and count of drops at locatioid = 258
# join on locationid with watermarking, count(*) from nyctaxi, min(temp) from temp stream





In [ ]:
# ASYNC EXAMPLE

# import asyncio

# async def worker(name):
#     print(name, "started")
#     for i in range(5):
#         print(name, "step", i)
#         await asyncio.sleep(0.5)  # yield time
#     print(name, "finished")

# async def main():
#     asyncio.create_task(worker("A"))
#     asyncio.create_task(worker("B"))
#     asyncio.create_task(worker("C"))

#     print("Main is doing something...")
#     await asyncio.sleep(3)  # yields → all tasks run
#     print("Main done")

# # asyncio.run(main())
# await main()



Main is doing something...
A started
A step 0
B started
B step 0
C started
C step 0
A step 1
B step 1
C step 1
A step 2
B step 2
C step 2
A step 3
B step 3
C step 3
A step 4
B step 4
C step 4
A finished
B finished
C finished
Main done
